In [19]:
from bs4 import BeautifulSoup as bs
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [20]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
soup = bs(r.content)

### function to get value from "td" tag

In [21]:
def get_content_value(row_data):
    if row_data.find('li'):  # Если есть список, то записываем данные в список
        return [li.get_text(" ", strip = True).replace("\xa0", " ") for li in row_data.find_all('li')]
    else:
        return row_data.get_text(" ", strip = True).replace("\xa0", " ") # иначе просто считываем и удаляем ненужный символ
    
      

### function to get a list of links to relevant movies

In [22]:
# ф-я чистовик. Сюа помещу готовый код
links_list = []
def get_links(html):
    r = requests.get(html)
    soup = bs(r.content)
    tables = soup.find_all('table', class_='wikitable sortable')[:-1]
    for row in tables:    # Открывается таблица
        tr = row.find_all('tr')  # кол-во строк в ней
        for i in range(1, len(tr)):    # открывается строка таблицы
            if(row.find_all('tr')[i].find_all('td')[1].find('a')):
                sub_tag = row.find_all('tr')[i]
                tag = sub_tag.find_all('td')[1].find('a')
                s1 = 'https://en.wikipedia.org'
                s2 = tag['href']
                link = s1 + s2   # Ссылка на страницу данного фильма
                links_list.append(link)
            else:
                continue
    return links_list

In [23]:
get_links('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
len(links_list)

436

### scraping information about each movie on the list and put it all  on a new list

In [24]:
movie_info_list = []
print(len(links_list))
def clean_tags(soup):
    for tag in soup.find_all(["sup", "span"]):
        tag.decompose()
clean_tags(soup)
for number, link in enumerate(links_list):
    req = requests.get(link)
    link_soup = bs(req.content)
    info_box = link_soup.find(class_="infobox vevent")
    info_rows = info_box.find_all('tr')
    movie_info = {}
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find('th').get_text(" ", strip = True)
        elif index == 1:
            continue
        elif (row.find('td') and row.find('th')):
            content_key = row.find('th').get_text(" ", strip = True)
            content_value = get_content_value(row.find('td'))         # значение словаря - текст в td
            movie_info[content_key] = content_value 
        else:
            continue
    movie_info_list.append(movie_info)

436


In [25]:
len(movie_info_list)

436

### Save/Load our data

In [26]:
import json
def save_data(title, data):
    with open(title, 'w', encoding = 'utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [27]:
def load_data(title):
    with open(title, encoding = 'utf-8') as f:
        return json.load(f)

In [28]:
save_data('disney_data_cleaned.json', movie_info_list)